# Análisis Comparativo de Modelos de PLN para la Clasificación de Sentimiento en Tweets
####(Punto 2 del trabajo: Aplicaciones de la Arquitectura Transformer al Pronóstico y Procesamiento de Lenguaje Natural)

In [1]:
# Montar Google Drive y cargar el dataset
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Mounted at /content/drive


In [7]:
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import json

# Cargar el dataset
dataset_path = '/content/drive/MyDrive/dataSecurncial/Punto2/twitter_training.csv.zip'
dataset = pd.read_csv(dataset_path)

# Utilizar la columna de texto y la de etiqueta de sentimiento
tweets = dataset.iloc[:, 3].astype(str)  # Convertir a string
etiquetas = dataset.iloc[:, 2].values

# Codificar las etiquetas
label_encoder = LabelEncoder()
encoded_etiquetas = label_encoder.fit_transform(etiquetas)

# Preprocesamiento de los datos
# Tokenización
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(tweets)


word_index = tokenizer.word_index

# Convertir textos a secuencias
sequences = tokenizer.texts_to_sequences(tweets)

# Padding
padded = pad_sequences(sequences, maxlen=200)

# Convertir etiquetas a categóricas
etiquetas_categoricas = tf.keras.utils.to_categorical(encoded_etiquetas)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_padded, test_padded, train_etiquetas, test_etiquetas = train_test_split(padded, etiquetas_categoricas, test_size=0.2)

# Cargar el modelo guardado
model = load_model('/content/drive/MyDrive/dataSecurncial/Punto2/modelo_rnn_twitter_sentiment.h5')

# Resumen del modelo (opcional)
model.summary()

# Función para predecir la etiqueta de un tweet
def predecir_tweet(texto):
    secuencia = tokenizer.texts_to_sequences([texto])
    padded_secuencia = pad_sequences(secuencia, maxlen=200)
    prediccion = model.predict(padded_secuencia)
    clase_predicha = label_encoder.inverse_transform([np.argmax(prediccion)])
    print("Predicción:", prediccion, "\nClase Predicha:", clase_predicha[0])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 200, 64)           24832     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 357380 (1.36 MB)
Trainable params: 357380 (1.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Ejemplo de uso: predecir el sentimiento de un nuevo tweet
tweet = "ice cream is delicious"
predecir_tweet(tweet)


1/1 [==============================] - 0s 154ms/step
Predicción: [[9.9832636e-01 1.9099700e-04 7.0900947e-04 7.7360758e-04]] 
Clase Predicha: Irrelevant


In [16]:
import json

# Suponiendo que `tokenizer` es tu objeto Tokenizer después de fit_on_texts

# Guardar el word_index del tokenizer
with open('/content/drive/MyDrive/dataSecurncial/Punto2/tokenizer.json', 'w') as f:
    json.dump(tokenizer.word_index, f)

# Guardar el label encoder
with open('/content/drive/MyDrive/dataSecurncial/Punto2/label_encoder.json', 'w') as f:
    json.dump(list(label_encoder.classes_), f)


In [12]:
import numpy as np
import pandas as pd

# Cargar el dataset
dataset_path = '/content/drive/MyDrive/dataSecurncial/Punto2/twitter_training.csv.zip'
dataset = pd.read_csv(dataset_path)

# Obtener la tercera columna
etiquetas = dataset.iloc[:, 2].values

# Contar el número de veces que aparece cada clase
etiquetas_unicas, etiquetas_counts = np.unique(etiquetas, return_counts=True)

# Imprimir las clases y sus conteos
for i in range(len(etiquetas_unicas)):
    print(f"Clase: {etiquetas_unicas[i]}, Conteo: {etiquetas_counts[i]}")


Clase: Irrelevant, Conteo: 12990
Clase: Negative, Conteo: 22542
Clase: Neutral, Conteo: 18318
Clase: Positive, Conteo: 20831


In [1]:
!pip install tensorflowjs

In [2]:
!tensorflowjs_converter --input_format keras /content/drive/MyDrive/dataSecurncial/Punto2/modelo_rnn_twitter_sentiment.h5 /content/drive/MyDrive/dataSecurncial/Punto2

2024-06-12 22:21:38.629345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 22:21:43.164786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
